In [2]:
from torch_geometric.datasets import MoleculeNet
import torch
import numpy as np
import best_config
from torch_geometric.data import DataLoader
import optuna
import models

In [3]:
class Engine:
    def __init__(self, model, model_params, optimizer, device):
        self.model = model
        self.model_params = model_params
        self.optimizer = optimizer
        self.device = device
        self.loss_fn = torch.nn.BCELoss()

    def train(self, loader):
        self.model.train()
        # Enumerate over the data
        final_loss = 0
        for batch in loader:
            # Use GPU
            batch.to(self.device)  
            # Reset gradients
            self.optimizer.zero_grad() 
            # Passing the node features and the connection info
            if self.model_params['has_edge_info']:
                pred = self.model(batch.x.float(),
                                batch.edge_attr.float(), 
                                batch.edge_index, 
                                batch.batch)
            else:
                pred = self.model(batch.x.float(), 
                                batch.edge_index, 
                                batch.batch)
            # Calculating the loss and gradients
            loss = self.loss_fn(pred, batch.y)
            final_loss += loss.item()
            loss.backward()
            # Update using the gradients
            self.optimizer.step()   
        return final_loss / len(loader)
    
    def evaluate(self, data_loader):
            self.model.eval()
            final_loss = 0
            for batch in data_loader:
                batch.to(self.device)
                if self.model_params['has_edge_info']:
                    pred = self.model(batch.x.float(),
                                    batch.edge_attr.float(), 
                                    batch.edge_index, 
                                    batch.batch)
                else:
                    pred = self.model(batch.x.float(), 
                                    batch.edge_index, 
                                    batch.batch)
                loss = self.loss_fn(pred, batch.y)  
                final_loss += loss.item()
            return final_loss / len(data_loader)

def run_training(params, save_model = False):
    data = MoleculeNet(root=".", name="bace")
    data = data.shuffle()
    #Specify device
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    
    model_params = params
    model_params['feature_size'] = data.num_features
    model_params["edge_dim"] = data[0].edge_attr.shape[1]
    model_params["has_edge_info"] = False
    
    model = eval('models.'+best_config.GLOBALPARAMETERS['model_this_run']+'(model_params)')
    print(model)

    model = model.to(device)

    # Wrap data in a data loader
    data_size = len(data)
    NUM_GRAPHS_PER_BATCH = model_params['batch_size']
    train_loader = DataLoader(data[:int(data_size * 0.8)], 
                        batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
    valid_loader = DataLoader(data[int(data_size * 0.8):int(data_size * 0.9)], 
                            batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
    test_loader = DataLoader(data[int(data_size * 0.9):], 
                            batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=model_params['learning_rate'])  

    eng = Engine(model, model_params, optimizer, device)

    best_loss = np.inf
    early_stopping_iter = 50
    early_stopping_counter = 0

    print("Starting training...")
    losses = []
    for epoch in range(100):
        loss = eng.train(train_loader)
        val_loss = eng.evaluate(valid_loader)
        losses.append(loss)
        if epoch % 10 == 0:
            print(f"Epoch {epoch} | Train Loss {loss} | Valid Loss {val_loss}")
        if val_loss < best_loss:
            best_loss = val_loss
            if save_model:
                torch.save(model.state_dict(), 'model.pt')
        else:
            early_stopping_counter +=1
        if early_stopping_counter > early_stopping_iter:
            break
    return best_loss

In [4]:
def objective(trial):
    params = {
        "model_layers": trial.suggest_int("model_layers", 1,7),
        "batch_size": trial.suggest_categorical('batch_size', [64,32,128]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 1e-6,1e-3),
        "model_embedding_size": trial.suggest_categorical("model_embedding_size",[32, 64, 128, 256,512,1024]),
        "model_gnn_dropout_rate": trial.suggest_uniform('model_gnn_dropout_rate', 0.01, 0.2),
        "model_linear_dropout_rate": trial.suggest_uniform('model_linear_dropout_rate', 0.01, 0.2),
        "model_dense_neurons": trial.suggest_categorical("model_dense_neurons",[32, 64, 128, 256])
    }
    best_loss = run_training(params, save_model=False)
    return best_loss

In [5]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials=1)

print('best_trial')
trial_ = study.best_trial

print(trial_.values)
print(trial_.params)

scores = run_training(trial_.params,save_model=True)

[I 2022-04-11 09:39:09,767] A new study created in memory with name: no-name-67240057-7ba3-4740-8b19-db9335593e82


GCN(
  (conv_layers): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
    (3): GCNConv(32, 32)
    (4): GCNConv(32, 32)
    (5): GCNConv(32, 32)
    (6): GCNConv(32, 32)
  )
  (initial_conv): GCNConv(9, 32)
  (gnn_dropout): Dropout(p=0.103304528314003, inplace=False)
  (linear_dropout): Dropout(p=0.038465025549285395, inplace=False)
  (linear): Linear(in_features=64, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


/home/rajeckidoyle/miniconda3/envs/my_torch_geometric/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Starting training...
Epoch 0 | Train Loss 0.6991735160350799 | Valid Loss 0.6972493529319763
Epoch 10 | Train Loss 0.6884873032569885 | Valid Loss 0.6793898344039917
Epoch 20 | Train Loss 0.6891699075698853 | Valid Loss 0.6880953907966614
Epoch 30 | Train Loss 0.687045156955719 | Valid Loss 0.6628091335296631
Epoch 40 | Train Loss 0.6785351872444153 | Valid Loss 0.6737695038318634
Epoch 50 | Train Loss 0.6712668120861054 | Valid Loss 0.6869352757930756


[I 2022-04-11 09:39:31,546] Trial 0 finished with value: 0.6324683427810669 and parameters: {'model_layers': 7, 'batch_size': 128, 'learning_rate': 0.0002341814653322955, 'model_embedding_size': 32, 'model_gnn_dropout_rate': 0.103304528314003, 'model_linear_dropout_rate': 0.038465025549285395, 'model_dense_neurons': 32}. Best is trial 0 with value: 0.6324683427810669.


best_trial
[0.6324683427810669]
{'model_layers': 7, 'batch_size': 128, 'learning_rate': 0.0002341814653322955, 'model_embedding_size': 32, 'model_gnn_dropout_rate': 0.103304528314003, 'model_linear_dropout_rate': 0.038465025549285395, 'model_dense_neurons': 32}
GCN(
  (conv_layers): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
    (3): GCNConv(32, 32)
    (4): GCNConv(32, 32)
    (5): GCNConv(32, 32)
    (6): GCNConv(32, 32)
  )
  (initial_conv): GCNConv(9, 32)
  (gnn_dropout): Dropout(p=0.103304528314003, inplace=False)
  (linear_dropout): Dropout(p=0.038465025549285395, inplace=False)
  (linear): Linear(in_features=64, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
Starting training...
Epoch 0 | Train Loss 0.688896757364273 | Valid Loss 0.6915474832057953
Epoch 10 | Train Loss 0.6876207590103149 | Valid Loss 0.6860975623130798
Epoch 20 | Train Loss 0.6850154280662537 | Valid Loss 0.6983514726161957
Ep

In [7]:
import pandas as pd

best_parameters = pd.DataFrame([trial_.params])
best_parameters["Best Loss"] = trial_.values
best_parameters.to_csv('/home/rajeckidoyle/Documents/Classification/BACE_Classification/model_benchmarks/hyperparameter_files/best_parameters/best_parameters.csv')